In [1]:
using Revise # FIXME
using MotionCaptureJointCalibration
using RigidBodyDynamics
using StaticArrays
using ValkyrieRobot
using ForwardDiff
using Ipopt

INFO: Recompiling stale cache file /home/twan/code/RigidBodyDynamics/lib/v0.6/MotionCaptureJointCalibration.ji for module MotionCaptureJointCalibration.


We'll first include some functionality for generation of synthetic measurement data:

In [2]:
include(joinpath(Pkg.dir("MotionCaptureJointCalibration"), "test", "synthetic_data_generation.jl"))

generate_pose_data (generic function with 2 methods)

Create a `Mechanism` (we'll use the NASA Valkyrie robot) and `MechanismState`:

In [3]:
T = Float64
val = Valkyrie()
mechanism = val.mechanism
remove_fixed_tree_joints!(mechanism)
state = MechanismState{T}(mechanism)

MechanismState{Float64, Float64, Float64, …}(…)

In [4]:
foot = findbody(mechanism, "leftFoot")
pelvis = findbody(mechanism, "pelvis")
markerbodies = [pelvis; foot]
scales = Dict(pelvis => 5., foot => 1.) # be careful with these; having them be different orders of magnitude can lead to numerical issues
correction_joints = collect(path(mechanism, pelvis, foot))
calibration_param_bounds = Dict(j => [(-0.05, 0.05)] for j in correction_joints)
free_joint_configuration_bounds = Dict(j => fill((-1., 1.), num_positions(j)) for j in tree_joints(mechanism) if isfloating(j))
srand(1)
ground_truth_marker_positions, measured_marker_positions = generate_marker_positions(markerbodies)
ground_truth_offsets = generate_joint_offsets(correction_joints, 1e-2)
ground_truth_pose_data, measured_pose_data = generate_pose_data(state, ground_truth_marker_positions, ground_truth_offsets, collect(keys(free_joint_configuration_bounds)))
;

In [5]:
problem = CalibrationProblem(mechanism, calibration_param_bounds, free_joint_configuration_bounds, measured_marker_positions, measured_pose_data);

In [8]:
problem

CalibrationProblem{Float64} with:
* 25 poses
* 6 calibration parameters
* 8 markers attached to 2 bodies



In [3]:
solver = IpoptSolver(print_level = 4, max_iter = 10000, derivative_test = "first-order", check_derivatives_for_naninf = "yes", tol = 1e-10)
# other useful options:
# hessian_approximation = "limited-memory"

problem = CalibrationProblem(mechanism, calibration_param_bounds, free_joint_configuration_bounds, measured_marker_positions, measured_pose_data)
result = solve(problem, solver)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

Starting derivative checker for first derivatives.


No errors detected by derivative checker.

Total number of variables............................:      362
                     variables with only lower bounds:        0
                variables with lower and upper bounds:      187
                     variables with only upper bounds:        0
Total number of equality constraints.................:      200
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
    

CalibrationResult{Float64}: Optimal, residual = 3.640506943053699e-9. Calibration parameters:
leftAnkleRoll: [-0.00407437]
leftHipRoll: [0.00903429]
leftKneePitch: [0.000381601]
leftAnklePitch: [-0.00977723]
leftHipPitch: [-0.0080456]
leftHipYaw: [0.00466544]

In [4]:
using RigidBodyTreeInspector
using DrakeVisualizer


Use "abstract type Widget end" instead.

Use "abstract type InputWidget{T}<:Widget end" instead.

Use "abstract type WidgetMod<:Widget end" instead.

Use "abstract type Container<:Widget end" instead.


In [7]:
DrakeVisualizer.any_open_windows() || (DrakeVisualizer.new_window(); sleep(1))
vis = Visualizer()[:valkyrie]
setgeometry!(vis, mechanism, parse_urdf(ValkyrieRobot.urdfpath(), mechanism; package_path = [ValkyrieRobot.packagepath()]))
state = MechanismState{T}(mechanism)

Bus::open: Connect ibus failed! 
IBusInputContext::createInputContext: no connection to ibus-daemon 


MechanismState{Float64, Float64, Float64, …}(…)

In [6]:
inspect!(state, vis, problem, result)

Interact.Options{:SelectionSlider,Bool}(1: "input" = false Bool , "", false, "Before cal", Interact.OptionDict(DataStructures.OrderedDict("Before cal"=>false,"After cal"=>true), Dict(false=>"Before cal",true=>"After cal")), Any[], Any[], true, "horizontal")

Interact.Slider{Int64}(3: "input-2" = 1 Int64 , "Pose number", 1, 1:25, "horizontal", true, "d", true)